# Pose Estimation from Videos
**Jin Yeom**  
jin.yeom@hudl.com

In [22]:
import sys
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.animation import ArtistAnimation
from IPython.display import HTML
from tqdm import tqdm_notebook as tqdm

In [2]:
try:
    sys.path.append('/openpose/build/python')
    from openpose import pyopenpose as op
except ImportError as e:
    print("Error: OpenPose library could not be found. Did you enable `BUILD_PYTHON` in CMake and have this Python script in the right folder?")
    raise e

In [3]:
params = dict(
    model_folder='/openpose/models/',
    number_people_max = 1
)

In [4]:
colors = [
    [255, 0, 0], 
    [255, 85, 0], 
    [255, 170, 0], 
    [255, 255, 0], 
    [170, 255, 0], 
    [85, 255, 0], 
    [0, 255, 0], 
    [0, 255, 85], 
    [0, 255, 170], 
    [0, 255, 255], 
    [0, 170, 255], 
    [0, 85, 255], 
    [0, 0, 255],
    [85, 0, 255], 
    [170, 0, 255],
    [255, 0, 255],
    [255, 85, 255]
]

# NOTE: this corresponds to COCO dataset
joint_pairs = [
    [0, 1], 
    [1, 3], 
    [0, 2], 
    [2, 4],
    [5, 6], 
    [5, 7], 
    [7, 9], 
    [6, 8], 
    [8, 10],
    [5, 11], 
    [6, 12], 
    [11, 12],
    [11, 13], 
    [12, 14], 
    [13, 15], 
    [14, 16]
]

In [5]:
# convert openpose keypoints (25) format to COCO keypoints (17) format
def op2coco(op_kpts):
    coco_kpts = []
    for i, j in enumerate([0, 16, 15, 18, 17, 5, 2, 6, 3, 7, 4, 12, 9, 13, 10, 14, 11]):
        score = op_kpts[j][-1]
        # if eye and ear keypoints score is lower, map it to mouth
        if score < 0.2 and j in [15, 16, 17, 18]:
            coco_kpts.append(op_kpts[0])
        else:
            coco_kpts.append(op_kpts[j])
    return np.stack(coco_kpts)

In [6]:
def with_pose(image, keypoints, keypoint_thresh=0.1):
    confidence = keypoints[:, 2:]
    coordinates = keypoints[:, :2]
    joint_visible = confidence[:, 0] > keypoint_thresh
    
    for color, (x, y, c) in zip(colors, keypoints):
        if c < keypoint_thresh:
            continue
        cv2.circle(image, (int(x), int(y)), 4, color, thickness=-1)
    
    for color, jp in zip(colors, joint_pairs):
        if joint_visible[jp[0]] and joint_visible[jp[1]]:
            pt0 = coordinates[jp, 0]
            pt1 = coordinates[jp, 1]
            pt0_0, pt0_1, pt1_0, pt1_1 = int(pt0[0]), int(pt0[1]), int(pt1[0]), int(pt1[1])
            cv2.line(image, (pt0_0, pt1_0), (pt0_1, pt1_1), color, 2)
            
    return image

In [7]:
# Starting OpenPose
op_wrapper = op.WrapperPython()
op_wrapper.configure(params)
op_wrapper.start()

In [20]:
def video2frames(filename, start_time, end_time, sx=0.5, sy=0.5, frame_skip=4):
    video = cv2.VideoCapture(filename)

    fps = video.get(cv2.CAP_PROP_FPS)
    start_frame = int(fps * start_time)  
    end_frame = int(fps * end_time)
    n_frames = end_frame - start_frame

    for i in tqdm(range(start_frame, end_frame, frame_skip)):
        video.set(cv2.CAP_PROP_POS_FRAMES, i)
        success, frame = video.read()
        height = int(sy * frame.shape[0])
        width = int(sx * frame.shape[1])
        frame = cv2.resize(frame, (width, height))  # make it smaller
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # to RGB
        yield frame

In [23]:
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10, 8), dpi=120)
ax0.set_axis_off()
ax1.set_axis_off()

ims = []
for i, frame in enumerate(video2frames('data/gd.mp4', 0, 225)):
    datum = op.Datum()
    datum.cvInputData = frame
    op_wrapper.emplaceAndPop([datum])
    keypoints = datum.poseKeypoints
    
    im0 = ax0.imshow(frame)
    if len(keypoints.shape):
        keypoints = op2coco(keypoints[0])
        im1 = ax1.imshow(with_pose(frame, keypoints))
    else:
        im1 = ax1.imshow(frame)
    ims.append([im0, im1])
    
anime = ArtistAnimation(fig, ims, interval=30, blit=True)
plt.close(fig=fig)
HTML(anime.to_html5_video())

RuntimeError: Requested MovieWriter (ffmpeg) not available

## References

- https://github.com/CMU-Perceptual-Computing-Lab/openpose
- https://github.com/esemeniuc/openpose-docker
- https://github.com/lxy5513/videopose